In [1]:
from google.colab import drive
drive.mount('/content/drive')
# 현재 디렉토리 설정
%cd /content/drive/My Drive/seoultech_ML/wildfire

Mounted at /content/drive
/content/drive/My Drive/seoultech_ML/wildfire


In [2]:
!pip install seaborn
!pip install scikit-learn

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from joblib import dump
from joblib import load
from itertools import combinations

## 데이터셋 설정

---
랜덤포레스트는 따로 전처리 및 스케일링 x

이상치에 덜 민감

In [4]:
# CSV 파일 로드
data = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')

In [5]:
### 데이터셋 준비 ###

# 학습 데이터
X_train = data.drop('class', axis=1)  # target 열을 제외한 데이터
y_train = data['class']  # 예측할 타겟 변수

# 검증 데이터
X_val = val.drop('class', axis=1)
y_val = val['class']

## RandomForest 정의

---
수치형 데이터셋 스케일링 할 필요 x

n_estimators와 max_features는 모델 성능에 큰 영향을 미치는 인자로, 이 두 매개변수를 조정하여 모델의 성능을 크게 개선할 수 있습니다.


min_samples_split, min_samples_leaf, max_depth 등은 모델의 과적합을 방지하기 위해 조정할 수 있는 인자입니다.


In [8]:
model = RandomForestClassifier(
    n_estimators=100,  # 트리의 개수. 더 많은 트리는 보통 더 좋은 성능을 의미하지만 계산 시간이 늘어납니다.
    criterion='entropy',  # 불순도를 측정하는 함수. 'gini'나 'entropy'를 사용할 수 있습니다.
    max_depth=None,  # 트리의 최대 깊이. 너무 깊어지면 과적합의 위험이 있습니다.
    min_samples_split=2,  # 노드를 분할하기 위해 필요한 최소 샘플 수.
    min_samples_leaf=1,  # 리프 노드가 가져야 하는 최소 샘플 수. 과적합을 제어하는 데 도움이 됩니다.
    max_features='auto',  # 최적의 분할을 찾기 위해 고려할 특성의 수. 'auto', 'sqrt', 'log2', int, float 등을 사용할 수 있습니다.
    max_leaf_nodes=None,  # 리프 노드의 최대 수. 이를 통해 트리의 형태를 제어할 수 있습니다.
    min_impurity_decrease=0.0,  # 노드를 분할할 때 감소해야 하는 불순도의 최소량.
    bootstrap=True,  # 부트스트랩(중복 허용 샘플링) 사용 여부. False로 설정하면 전체 데이터셋을 사용합니다.
    oob_score=True,  # True로 설정하면 트레이닝 데이터의 일부가 아닌 샘플을 사용하여 모델의 일반화 정확도를 평가합니다.
    n_jobs=None,  # 학습과 예측에 사용할 작업 수. '-1'로 설정하면 모든 프로세서를 사용합니다.
    random_state=42,  # 결과의 재현 가능성을 위한 랜덤 시드 값.
    class_weight={0: 1, 1: 10},  # 클래스 가중치. 'balanced', 'balanced_subsample', 딕셔너리 형태 또는 'None'을 사용할 수 있습니다.
    ccp_alpha=0.0  # 비용 복잡도 가지치기(Cost Complexity Pruning)의 복잡도 매개변수. 과적합을 방지하는 데 사용됩니다.
)

In [9]:
### 모델 학습 ###

model.fit(X_train, y_train)
train_pred = model.predict(X_train)

# 학습 정확도 계산
train_accuracy = accuracy_score(y_train, train_pred)
print("Training Accuracy:", train_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Training Accuracy: 1.0


In [10]:
### 평가지표 ### - val(OA)
### 추론 ###
predictions = model.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f'Accuracy (OA): {accuracy}')

Accuracy (OA): 0.9808115759672853


In [11]:
### 혼동행렬 및 정밀도,재현율 출력 ###
conf_matrix = confusion_matrix(y_val, predictions)
precision = precision_score(y_val, predictions)
recall = recall_score(y_val, predictions)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Confusion Matrix:\n{conf_matrix}')

Precision: 0.9233576642335767
Recall: 0.863481228668942
Confusion Matrix:
[[2865   21]
 [  40  253]]


## 변수 중요도 평가

In [12]:
# 변수 중요도 평가
importances = model.feature_importances_

# 변수 이름과 중요도를 딕셔너리로 묶기
feature_importances = {name: score for name, score in zip(X_train.columns, importances)}

# 중요도에 따라 변수를 정렬
sorted_feature_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)

# 상위 n개 변수와 중요도 출력
for feature, importance in sorted_feature_importances[:20]:
    print(f"{feature}: {importance}")

# 중요도가 높은 변수의 인덱스를 얻기
indices = np.argsort(importances)[::-1]

# 중요도가 높은 변수만 선택
selected_features = X_train.columns[indices[:17]]  # n은 선택할 변수의 수
selected_features

ch06_07: 0.10920985595978523
ch05_07: 0.10034648020632683
bt07_13r: 0.09298401244885322
bt07_13: 0.08092106372737555
bt07_11r: 0.06244379065310728
bt07_14r: 0.06125411726652207
bt07_11: 0.059847500047615146
bt07_12r: 0.058308888413874854
bt07_15r: 0.05163448624865121
ch07_13: 0.050290245093231024
bt13_15: 0.046112035016650475
ch12_15: 0.04592903751026598
bt13_14r: 0.036552213737391204
bt13_15r: 0.03573626828334666
bt07_08r: 0.026411877131546205
bt07_10r: 0.023942019041735565
bt07_16r: 0.0205753086594184
bt11_15r: 0.02049713427837727
bt07_09r: 0.017003666275925967


Index(['ch06_07', 'ch05_07', 'bt07_13r', 'bt07_13', 'bt07_11r', 'bt07_14r',
       'bt07_11', 'bt07_12r', 'bt07_15r', 'ch07_13', 'bt13_15', 'ch12_15',
       'bt13_14r', 'bt13_15r', 'bt07_08r', 'bt07_10r', 'bt07_16r'],
      dtype='object')